In [ ]:
%load_ext lab_black

Step: Load data

In [ ]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

In [ ]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [ ]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [ ]:
import os

script_path = os.path.abspath("production_injury_data.py")

src = ScriptRunConfig(source_directory=".", script=script_path, environment=environment)

In [ ]:
# Create a ScriptRunConfig for your script
src = ScriptRunConfig(
    source_directory=".", script="production_injury_data.py", environment=environment
)

In [ ]:
# Create a PythonScriptStep
step = PythonScriptStep(
    name="scrapper_nfl_injury",
    script_name="production_injury_data.py",
    compute_target=compute_target,
    runconfig=src.run_config,
)

In [ ]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [ ]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="scrapper_nfl_injury_pipeline")

In [ ]:
from datetime import time

recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="15:02")
schedule = Schedule.create(
    workspace,
    name="scrapper_nfl_injury_schedule",
    pipeline_id=published_pipeline.id,
    experiment_name="scrapper_nfl_injury_experiment",
    recurrence=recurrence,
)

In [ ]:
schedules = Schedule.list(workspace)
for s in schedules:
    print(f"Schedule name: {s.name}, Status: {s.status}, Pipeline ID: {s.pipeline_id}")

In [63]:
published_pipeline.id

'd7e805cc-941e-4477-bff1-c6a4ad480fa6'

In [ ]:
from azureml.pipeline.core import PublishedPipeline

# Specify the name of the pipeline you are interested in
pipeline_name = "scrapper_nfl_injury_pipeline"

# Get the list of published pipelines in the workspace
published_pipelines = PublishedPipeline.list(workspace)
published_pipelines

In [67]:
from azureml.pipeline.core.run import PipelineRun

# Replace with the latest pipeline run ID
run_id = "c090033d-0d4f-4bb6-9bef-94fabbe41107"

# Get the specific pipeline run
pipeline_run = PipelineRun.get(workspace=workspace, run_id=run_id)

ErrorResponseException: (NotFound) Response status code does not indicate success: 404 (Not Found).
Failed to get item from container: Experiment, id: c090033d-0d4f-4bb6-9bef-94fabbe41107, partition e9d35628-3c1a-4162-87e6-c51253db20bc, error code NotFound

In [ ]:


# Access and print the details of the run
print(f"Pipeline Run ID: {pipeline_run.id}")
print(f"Status: {pipeline_run.status}")
print(f"Start Time: {pipeline_run.start_time}")
print(f"End Time: {pipeline_run.end_time}")
# Print other relevant details or extract specific metrics or logs


In [66]:
from azureml.core import Experiment

experiments = Experiment.list(workspace)
for experiment in experiments:
    print(f"Experiment name: {experiment.name}, ID: {experiment.id}")

Experiment name: nfl-injury-scraper-exp, ID: c090033d-0d4f-4bb6-9bef-94fabbe41107
Experiment name: nfl-scraper-exp, ID: dc123887-4794-42d0-8a08-3faea50a89f6
Experiment name: nfl-injury-scraper, ID: 170bc2ad-69af-43ac-81d5-f421b0b51859
Experiment name: nfl_injury_scraper_experiment, ID: 52a91862-feff-4e2e-85b5-4a8cb89ea45e
Experiment name: scrapper_nfl_injury_experiment, ID: 894f86ed-c0dc-49fb-a870-39c729620bd6
Experiment name: scrapper_nfl_injury_experiment_20230514-185421, ID: 96324f72-4e43-4a93-b1fa-ad72ba304376
